# Guide
- requests.get()의 **[파라미터 관련](#requests.get(URL))**
    - **user agent 조회**
        - 크롬 개발자 도구 콘솔에서 자바스크립트 navigator객체로 조회
        - `navigator.userAgent` 
    - 튜플로 전달 가능
        -  \[('key1', 'value1'), ('key1', 'value2')\]
    - 같은 이름으로 여러개 전달시
        - payload_dict = {'key1': \['value1', 'value2'\]}
        
- requests.post()의 **post로 파일업로드 (https://requests.readthedocs.io/en/latest/user/quickstart/#post-a-multipart-encoded-file) 참고**
```python
url = 'https://httpbin.org/post'
files = {'file': open('report.xls', 'rb')}
# 파일명, 파일 content 타입, header 정보등 설정 가능
# files = {'file': ('report.xls', open('report.xls', 'rb'), 'application/vnd.ms-excel', {'Expires': '0'})}
# multipart/form-data 인코딩 처리 (post 로 전달하면 됨)
r = requests.post(url, files=files)
```

```python
files = {
    'file1': open('img1.jpg', rb), 
    'file2': open('img2.jpg', rb), 
    'file3': ('img3.jpg', open('img3.jpg'),'image/jpg', {'Expires':0}
# multipart/form-data 인코딩 처리 (post 로 전달하면 됨)
res = requests.post('http://httphin.org/post', files=files)
```

- **[이미지등 binary데이터를 받은 경우](#Response객체)**
```python
img = res.content
with open('저장할 파일명', 'wb') as f:
    f.write(img)
```    
- **http://httpbin.org 란**
    - http요청 테스트 사이트

 # urllib 패키지를 이용한 URL 다루기
- urllib 패키지
     - URL 작업을 위한 여러 모듈을 모은 패키지
- 주요모듈
    - urllib.request: URL을 열고 요청을 위한 모듈
    - urllib.parse: URL 구문 분석을 위한 모듈
    - urllib.robotparser: robots.txt 파일을 구문 분석하기 위한 모듈

## URL (Uniform Resource Locator) 이란
- 네트워크 상의 자원(html문서, 이미지 등등) 이 어디에 있는지 그 위치를 식별하기 위한 규약
    - 인터넷상의 웹페이지등을 찾기 위한 주소를 말한다.
- [위키백과](https://ko.wikipedia.org/wiki/URL)

### URL의 구성요소
- 구문
    - ` scheme:[//[user:password@]host[:port]][/]path[?query][#fragment]`
    
- 스키마(schema): 통신 방식
- 호스트(host): 서버 주소
- 포트번호(port): 서버 프로세스를 구분하는 식별번호
- 패스(path): 서버에서 문서의 위치
- 쿼리(query): 문서에 전달하는 추가 정보

### URL 예

```
https://search.naver.com:80/search.naver?sm=top_hty&fbm=1&ie=utf8&query=scraping
```

- 스키마(scheme): `https://`
- 호스트(host): `search.naver.com`
- 포트번호(port): `80`. 웹브라우저들은 port 번호 생략시 80으로 전송된다.
- 자원경로(path): `search.naver`
- 쿼리스트링(query): `?sm=top_hty&fbm=1&ie=utf8&query=scraping`

## urllib.parse 모듈을 이용한 URL 다루기
- 파이썬 내장모듈로 url과 관련된 다양한 기능을 제공하는 모듈
    - url 구문 분석
    - url 경로 합치기    

### url 분석 (parsing)
- urllib.parse 모듈의 urlparse()함수 이용
- **urlparse(분석할 URL) : ParseResult**
- ParseResult의 속성을 이용해 url 구성요소 조회
    - scheme, hostname, port, path, query

In [52]:
from urllib import parse
url = 'https://search.naver.com:80/search.naver?sm=top_hty&fbm=1&ie=utf8&query=scraping'
p = parse.urlparse(url)

In [53]:
# G: url에 없으면 안나온다. 알아서 80 붙이지 않는다.
p.scheme, p.hostname, p.port, p.path, p.query

'https'

### url 합치기
크롤링시 같은 사이트의 여러 다른 자원을 조회하는 경우가 많다. 이럴때 host는 동일하고 path 이후가 바뀐다. 그래서 중복되는 url을 base url 로 지정하고 바뀌는 부분만 붙여서 url을 완성하면 편리하다.
- **parse.urljoin(base, url)**
    - base+'/'+url 형태로 합쳐 준다.

In [1]:
from urllib import parse

base_url = 'http://main.co.kr/'
path1 = '/news/sports.html'
path2 = '/news/it.html'
path3 = '/shopping/computer.html'
print(parse.urljoin(base_url, path1))
print(parse.urljoin(base_url, path2))
print(parse.urljoin(base_url, path3))

http://main.co.kr/news/sports.html
http://main.co.kr/news/it.html
http://main.co.kr/shopping/computer.html


In [2]:
paths = [path1, path2, path3]
for path in paths:
    url = parse.urljoin(base_url, path)
    print(url, "처리")

http://main.co.kr/news/sports.html 처리
http://main.co.kr/news/it.html 처리
http://main.co.kr/shopping/computer.html 처리


# requests 모듈을 이용한 웹 요청
- [Requests 홈페이지](https://requests.kennethreitz.org/en/master/)
- **HTTP 요청을 처리하는 파이썬 패키지**
- get/post 방식 모두를 지원하며 쿠키, 헤더정보등을 HTTP의 다양한 요청처리를 지원한다.
- 내장 라이브러리가 아니므로 인스톨이 필요
    - 아나콘다 배포판에는 내장되어 있어 별도의 인스톨이 필요없다.
    - `pip install requests`
    - `conda install -c conda-forge requests`

## requests 코딩 패턴
1. requests 모듈 import
2. get()/post() 함수를 이용해 특정 url의 페이지 요청
3. 응답받은 내용을 처리.
    - text(HTML)은 BeautifulSoup에 넣어 parsing
    - binary 파일의 경우 파일출력 처리

# Guide: 일단 만들어 보기

In [1]:
from bs4 import BeautifulSoup
import requests

url = 'http://www.pythonscraping.com/pages/warandpeace.html'
res = requests.get(url) #응답 결과를 반환(requests.models.Response)
soup = BeautifulSoup(res.text)

# 초록색 글 (class=green) 조회 (Guide: 크롬개발자 도구로 확인)
greens = soup.find_all('span', class_='green')
txt_list = [tag.get_text().strip().replace('\n', ' ') for tag in greens]
txt_list

['Anna Pavlovna Scherer',
 'Empress Marya Fedorovna',
 'Prince Vasili Kuragin',
 'Anna Pavlovna',
 'St. Petersburg',
 'the prince',
 'Anna Pavlovna',
 'Anna Pavlovna',
 'the prince',
 'the prince',
 'the prince',
 'Prince Vasili',
 'Anna Pavlovna',
 'Anna Pavlovna',
 'the prince',
 'Wintzingerode',
 'King of Prussia',
 'le Vicomte de Mortemart',
 'Montmorencys',
 'Rohans',
 'Abbe Morio',
 'the Emperor',
 'the prince',
 'Prince Vasili',
 'Dowager Empress Marya Fedorovna',
 'the baron',
 'Anna Pavlovna',
 'the Empress',
 'the Empress',
 "Anna Pavlovna's",
 'Her Majesty',
 'Baron Funke',
 'The prince',
 'Anna Pavlovna',
 'the Empress',
 'The prince',
 'Anatole',
 'the prince',
 'The prince',
 'Anna Pavlovna',
 'Anna Pavlovna']

## 요청 함수
- get(): GET방식 요청
- post(): POST방식 요청

## requests.get(URL)
- **GET 방식 요청**
- **주요 매개변수**
    - params: 요청파라미터를 dictionary로 전달
    - headers: HTTP 요청 header를 dictionary로 전달
        - 'User-Agent', 'Referer' 등 헤더 설정
    - cookies: 쿠키정보를 전달
- **반환값(Return Value)**
    - Response

## requests.post(URL)
- **POST 방식 요청**
- **주요 매개변수**
    - datas : 요청파라미터를 dictionary로 전달
    - files : 업로드할 파일을 dictionary로 전달
        - key: 이름, value: 파일과 연결된 InputStream(TextIOWrapper)
    - headers: HTTP 요청 header를 dictionary로 전달
        - 'User-Agent', 'Referer' 등 헤더 설정
    - cookies: 쿠키정보를 전달
- **반환값(Return Value)**
    - Response    

### 요청파라미터 전달
- 요청파라미터란?
    - 서버가 일하기 위해 클라이언트로 부터 받아야하는 값들
    - `name=value` 형태이며 여러개일 경우 `&`로 연결해서 전송됨
- Get 요청시 queryString 으로 전달
    - querystring : URL 뒤에 붙여서 전송한다.
    - ex) https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=python
    - requests.get() 요청시 
        1. url 뒤에 querystring으로 붙여서 전송
        2. dictionary 에 name=value 형태로 만들어 매개변수 params에 전달
- Post 요청시 요청정보의 body에 넣어 전달

> ### HTTP 요청 헤더(Request Header)
> HTTP 요청시 웹브라우저가 client의 여러 부가적인 정보들을 Key-Value 쌍 형식으로 전달한다.
> - accept: 클라이언트가 처리가능한 content 타입 (Mime-type 형식으로 전달)
> - accept-language: 클라이언트가 지원하는 언어(ex: ko, en-US)
> - host: 요청한 host 
> - user-agent: 웹브라우저 종류

## Response객체 - 응답데이터
- get()/post() 의 요청에 대한 서버의 응답 결과를 Response에 담아 반환
    - Response의 속성을 이용해 응답결과를 조회
- 주요속성
    - url: 응답 받은(요청한) url 
    - status_code: HTTP 응답 상태코드
    - headers: 응답 header 정보를 dictionary로 반환
- **응답 결과 조회**
    - text: 응답내용을 
    - content: 응답내용(응답결과가 binary 일 경우 - image, 동영상등)
    - json()
        - 응답 결과가 JSON 인 경우 dictionary로 변환해서 반환

> ### JSON(JavaScript Object Notation)
> key-value 형태 또는 배열 형태의 text이며 이 기종간 데이터 교환에 많이 사용된다. 자바스크립트 언어에서 Object와 array를 생성하는 문법을 이용해 만듬. 
- [JSON 공식사이트](http://json.org)
>
> ### json 모듈
> JSON 형식 문자열을 다루는 모듈
> - json.loads(json문자열)
>    - JSON 형식 문자열을 dictionary로 변환
> - json.dumps(dictionary)
>    - dictionary를 JSON 형식 문자열로 변환

> ### HTTP 응답 상태코드
> - https://developer.mozilla.org/ko/docs/Web/HTTP/Status 
- 2XX: 성공
    - 200: OK
- 3XX: 다른 주소로 이동 (이사)
    - 300번대이면 자동으로 이동해 준다. 크롤링시는 볼일이 별로 없다.
- 4XX: 클라이언트 오류 (사용자가 잘못한 것)
  - 404: 존재하지 않는 주소
- 5XX: 서버 오류 (서버에서 문제생긴 것)
  - 500: 서버가 처리방법을 모르는 오류
  - 503: 서버가 다운 등의 문제로 서비스 불가 상태

### Get 방식 요청 예제

In [3]:
import requests
from pprint import pprint
from urllib import parse
base_url = 'http://httpbin.org/{}'
# base_url = 'http://httpbin.org
# url = parse.urljoin(base_url, 'get')
url = base_url.format('get')
print(url)
#같은 이름으로 여러개 전송
params = {
    'name':['hong','lee'],
    'age':20
}
# params = (('name','lee'),('name','kim'), ('age',20))

params = {
    'name':['lee','kim'],
    'age':20
}
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'Test':'my-test'  
}
cookies = {'c1':'a100','c2':'a200'}
res = requests.get(url, params=params, headers=headers, cookies=cookies)
# print(res.text)
print(type(res.json()))
pprint(res.json())


http://httpbin.org/get
<class 'dict'>
{'args': {'age': '20', 'name': ['lee', 'kim']},
 'headers': {'Accept': '*/*',
             'Accept-Encoding': 'gzip, deflate, br',
             'Cookie': 'c1=a100; c2=a200',
             'Host': 'httpbin.org',
             'Test': 'my-test',
             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                           'AppleWebKit/537.36 (KHTML, like Gecko) '
                           'Chrome/111.0.0.0 Safari/537.36',
             'X-Amzn-Trace-Id': 'Root=1-643caaa1-185104b755b2571715db4d7c'},
 'origin': '222.112.208.66',
 'url': 'http://httpbin.org/get?name=lee&name=kim&age=20'}


In [2]:
result = res.json()
result.get('args').get('age')
result.get('url')

'http://httpbin.org/get?name=lee&name=kim&age=20'

In [3]:
# image
import requests
url = 'http://httpbin.org/image/png'
res = requests.get(url)
print(res.status_code)
if res.status_code == 200:
    with open('image.jpg', 'wb') as f:
        f.write(res.content)

200


### Post 요청 예

In [5]:
import requests
from pprint import pprint
from urllib import parse
base_url = 'http://httpbin.org'

# url = parse.urljoin(base_url, 'post')
url = "http://httpbin.org/post"
params = {
    'name':['hong','lee'],
    'age':20
}
# params = (('name','kim'),('name','kim'), ('age',20))
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
    'Test':'my-test'  
}
cookies = {'c1':'a100','c2':'a200'}
print(params)
res = requests.post(url, data=params, headers=headers, cookies=cookies)
# print(res)
#파일업로드
# res = requests.post(url, data=params, headers=headers,files={'file':open('02_requests.ipynb', 'rb')}, cookies=cookies)
print(res.text)
print(type(res.json()))
pprint(res.json())

{'name': ['hong', 'lee'], 'age': 20}
{
  "args": {}, 
  "data": "", 
  "files": {
    "file": "{\n \"cells\": [\n  {\n   \"cell_type\": \"markdown\",\n   \"metadata\": {},\n   \"source\": [\n    \"# Guide\\n\",\n    \"- requests.get()\uc758 **[\ud30c\ub77c\ubbf8\ud130 \uad00\ub828](#requests.get(URL))**\\n\",\n    \"    - **user agent \uc870\ud68c**\\n\",\n    \"        - \ud06c\ub86c \uac1c\ubc1c\uc790 \ub3c4\uad6c \ucf58\uc194\uc5d0\uc11c \uc790\ubc14\uc2a4\ud06c\ub9bd\ud2b8 navigator\uac1d\uccb4\ub85c \uc870\ud68c\\n\",\n    \"        - `navigator.userAgent` \\n\",\n    \"    - \ud29c\ud50c\ub85c \uc804\ub2ec \uac00\ub2a5\\n\",\n    \"        -  \\\\[('key1', 'value1'), ('key1', 'value2')\\\\]\\n\",\n    \"    - \uac19\uc740 \uc774\ub984\uc73c\ub85c \uc5ec\ub7ec\uac1c \uc804\ub2ec\uc2dc\\n\",\n    \"        - payload_dict = {'key1': \\\\['value1', 'value2'\\\\]}\\n\",\n    \"        \\n\",\n    \"- requests.post()\uc758 **post\ub85c \ud30c\uc77c\uc5c5\ub85c\ub4dc (https://requests.r

In [1]:
# import json
# dic = json.loads(res.text)
# print(dic['headers'])
# print('-'*20)
# j = json.dumps(dic)
# print(j)

### 응답결과(Response) 조회

In [8]:
import requests
url = 'http://www.pythonscraping.com/pages/warandpeace.html'
res = requests.get(url) #응답 결과를 반환(requests.models.Response)
#print(res.text)

In [9]:
#요청 url (응답을 받은 url)
res.url 

'https://www.pythonscraping.com/pages/warandpeace.html'

In [10]:
# 응답 상태코드
res.status_code

200

In [11]:
pprint(res.headers) #이거 자체가 Dictionary(requests.structures.CaseInsensitiveDict) -> 이건 JSON 변환안됨 (json.dumps(res.headers) 에러)

{'Server': 'nginx', 'Date': 'Tue, 11 Apr 2023 08:47:40 GMT', 'Content-Type': 'text/html', 'Last-Modified': 'Sat, 09 Jun 2018 19:15:59 GMT', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'ETag': 'W/"5b1c276f-2dcb"', 'X-Powered-By': 'PleskLin', 'Content-Encoding': 'br'}


In [65]:
# 응답 header 정보 조회
for key in res.headers:
    print(key, res.headers[key], sep=' : ')

Date : Sun, 20 Oct 2019 09:54:31 GMT
Server : Apache
Last-Modified : Sat, 09 Jun 2018 19:15:59 GMT
ETag : "4121bd1-2dcb-56e3a58bcb54a"
Accept-Ranges : bytes
Content-Length : 11723
Cache-Control : max-age=1209600
Expires : Sun, 03 Nov 2019 09:54:31 GMT
Keep-Alive : timeout=5, max=100
Connection : Keep-Alive
Content-Type : text/html


In [66]:
# 쿠키조회
res.cookies

<RequestsCookieJar[]>

In [67]:
# 인코딩 방식 조회
res.encoding

'ISO-8859-1'

In [17]:
# 내용을 text로 조회
print(res.text[:1000] )

<html>
<head>
<style>
.green{
	color:#55ff55;
}
.red{
	color:#ff5555;
}
#text{
	width:50%;
}
</style>
</head>
<body>
<h1>War and Peace</h1>
<h2>Chapter 1</h2>
<div id="text">
"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>"
<p/>
It was in July, 1805, and the speaker was the well-known <span class="green">Anna
Pavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya
Fedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man
of high rank and importance, who was the first t

In [69]:
# 내용을 binary로 조회 -> image등은 이것으로 받는다.
res.content[:1000]

b'<html>\n<head>\n<style>\n.green{\n\tcolor:#55ff55;\n}\n.red{\n\tcolor:#ff5555;\n}\n#text{\n\twidth:50%;\n}\n</style>\n</head>\n<body>\n<h1>War and Peace</h1>\n<h2>Chapter 1</h2>\n<div id="text">\n"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don\'t tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy \'faithful slave,\' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.</span>"\n<p/>\nIt was in July, 1805, and the speaker was the well-known <span class="green">Anna\nPavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya\nFedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man\nof high ran

## naver 검색

In [72]:
url = 'https://search.naver.com/search.naver'
params = {'sm':'top_hty', 'fbm':1, 'ie':'utf8', 'query':'python'}
res = requests.get(url, params=params)
res.url

'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=python'

In [73]:
print(res.text[:1500])

<!doctype html> <html lang="ko"> <head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=2.0"> <meta property="og:title" content="python : 네이버 통합검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="'python'의 네이버 통합검색 결과입니다."> <meta name="description" lang="ko" content="'python'의 네이버 통합검색 결과입니다."> <title>python : 네이버 통합검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_140327.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_191010.css"> <link rel="stylesheet" type="text/css" 

In [19]:
import requests
from bs4 import BeautifulSoup 
import re

# 뉴스기사 하나에서 제목, 입력일자, 내용 추출
url = 'https://news.v.daum.net/v/20191025171501898'
# url = 'https://news.v.daum.net/v/20191025103559221'
# url = 'https://news.v.daum.net/v/20200722113746430'
res = requests.get(url)

if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'lxml')

    # 찾을 때 ctr + f 로 해서 태그의 class나 id가 중복되어 있는 지 찾아본다.
    #제목 : h3.tit_view
    #기자: span.info_view > span.txt_info:nth-child(1) # 가끔 기자이름이 없는 경우도 있다. 
    #작성일: span.info_view > span.txt_info:nth-child(2)
    #수정일: span.info_view > span.txt_info:nth-child(3)
    #뉴스내용: div.news_view

    #제목 가져오기
    import re
#     title_tag = soup.select_one(re.compile(r'h\d.tit_view')) #안되네?
    title_tag = soup.select_one('h3.tit_view')
    info1_tag = soup.select_one('span.info_view > span.txt_info:nth-child(1)')
    info2_tag = soup.select_one('span.info_view > span.txt_info:nth-child(2)')
    info3_tag = soup.select_one('span.info_view > span.txt_info:nth-child(3)')
    news_tag = soup.select_one('div.news_view')


    print('제목:',title_tag.text.strip())
    print('기자:',info1_tag.text.strip())
    print('입력,수정일자:',info2_tag.text.strip(), info3_tag.text.strip())
    p = re.compile('\s+')
    print(p.sub(' ', news_tag.text))
    # content = re.sub('\n+', '\n', news.text)
    # print(content)
else:
    print("연결 실패 - %d" % res.status_code)    

제목: [Science] 행성? 블랙홀?..태양계 아홉번째 '행성9'의 정체를 밝혀라
기자: 송경은
입력,수정일자: 입력 2019.10.25. 17:15 수정 2019.10.25. 19:45
 '행성9'의 미스터리행성9 존재 어떻게 발견했나지구 직경 3.7배..질량은 5~15배타원형 궤도로 공전주기 1만~2만년태양계 외곽 소천체 궤도 특정방향 쏠려해왕성 바깥 '행성9'의 중력 작용 주장행성이 아닌 원시블랙홀일까138억년 전 빅행 직후 수초 내 탄생국지적으로 밀도 높아져서 형성손바닥 크기거나 지구보다 클 수도블랙홀 '감마선' 포착 땐 규명 가능 [사진 제공 = 게티이미지뱅크] 2014년 3월, 태양계에서 아홉 번째 행성을 새롭게 발견했다는 미국 과학자들 연구 결과가 과학저널 '네이처'에 실렸다. 이 논문은 발표 직후 천문학계를 뜨겁게 달궜다. 한때 태양계의 아홉 번째 행성이었던 명왕성이 2006년 행성 지위를 잃고 왜소행성으로 분류된 뒤 처음으로 아홉 번째 행성 후보가 나왔기 때문이다. 과학자들은 이 정체불명의 천체를 '행성9(planet 9)'라고 불렀다. 이후 아직까지 행성9가 태양계 아홉 번째 행성이라는 점을 뒷받침할 만한 직접적인 관측 근거는 없는 상태다. 행성9 정체에 대한 학술적 논쟁이 끊이지 않는 이유다. 이런 가운데 최근 행성9가 행성이 아니라 138억년 전 우주 초기에 형성된 '원시 블랙홀(PBH)'일 수 있다는 새로운 주장이 나와 화제를 모으고 있다. 제이컵 숄츠 영국 더럼대 교수와 제임스 언윈 미국 일리노이대 교수 연구진은 논문 초고 공개 사이트인 '아카이브(arXiv.org)'에 공개한 논문에서 "태양계 아홉 번째 행성 후보로 알려진 행성9가 원시 블랙홀일 가능성이 있다"며 "기존 예상보다 원시 블랙홀이 지구 가까이에 있을 수 있다"고 지난달 24일 밝혔다. 원시 블랙홀은 138억년 전 우주 대폭발(빅뱅) 직후 수초 내에 탄생한 상대적으로 작은 규모 블랙홀을 말한다. 영국 이론물리학자인 스티븐 호킹 박사가 1974년 처음 그 개념을 제시했다

In [17]:
import re
p = re.compile('\s+')
new_txt = p.sub(' ', news_tag.text)


In [18]:
new_txt

' \'행성9\'의 미스터리행성9 존재 어떻게 발견했나지구 직경 3.7배..질량은 5~15배타원형 궤도로 공전주기 1만~2만년태양계 외곽 소천체 궤도 특정방향 쏠려해왕성 바깥 \'행성9\'의 중력 작용 주장행성이 아닌 원시블랙홀일까138억년 전 빅행 직후 수초 내 탄생국지적으로 밀도 높아져서 형성손바닥 크기거나 지구보다 클 수도블랙홀 \'감마선\' 포착 땐 규명 가능 [사진 제공 = 게티이미지뱅크] 2014년 3월, 태양계에서 아홉 번째 행성을 새롭게 발견했다는 미국 과학자들 연구 결과가 과학저널 \'네이처\'에 실렸다. 이 논문은 발표 직후 천문학계를 뜨겁게 달궜다. 한때 태양계의 아홉 번째 행성이었던 명왕성이 2006년 행성 지위를 잃고 왜소행성으로 분류된 뒤 처음으로 아홉 번째 행성 후보가 나왔기 때문이다. 과학자들은 이 정체불명의 천체를 \'행성9(planet 9)\'라고 불렀다. 이후 아직까지 행성9가 태양계 아홉 번째 행성이라는 점을 뒷받침할 만한 직접적인 관측 근거는 없는 상태다. 행성9 정체에 대한 학술적 논쟁이 끊이지 않는 이유다. 이런 가운데 최근 행성9가 행성이 아니라 138억년 전 우주 초기에 형성된 \'원시 블랙홀(PBH)\'일 수 있다는 새로운 주장이 나와 화제를 모으고 있다. 제이컵 숄츠 영국 더럼대 교수와 제임스 언윈 미국 일리노이대 교수 연구진은 논문 초고 공개 사이트인 \'아카이브(arXiv.org)\'에 공개한 논문에서 "태양계 아홉 번째 행성 후보로 알려진 행성9가 원시 블랙홀일 가능성이 있다"며 "기존 예상보다 원시 블랙홀이 지구 가까이에 있을 수 있다"고 지난달 24일 밝혔다. 원시 블랙홀은 138억년 전 우주 대폭발(빅뱅) 직후 수초 내에 탄생한 상대적으로 작은 규모 블랙홀을 말한다. 영국 이론물리학자인 스티븐 호킹 박사가 1974년 처음 그 개념을 제시했다. 이처럼 원시 블랙홀은 항성(별)들이 출현하기 훨씬 전인 우주 초기에 탄생한 만큼 지구에서 아주 멀리 떨어진 심(深)우주에 존재할 것으로 여겨져왔다. 하지만 만약 

In [30]:
# %load daumnews.py
%%writefile daumnews.py

import re
import requests
from bs4 import BeautifulSoup

def get_daumnews_info(url):
    """
    [파라미터]
        url : 다음 기사 url
    [반환값]
        tuple: (제목, 기자이름, 입력일, 기사내용)
    """
    res = requests.get(url)
    if res.status_code==200:
        soup = BeautifulSoup(res.text, 'html.parser')
        title_tag = soup.select_one('h3.tit_view')
        reporter_tag = soup.select_one('span.info_view span.txt_info:nth-child(1)')
        input_date_tag = soup.select_one('span.info_view span.txt_info:nth-child(2)')
        news_tag = soup.select_one('div#harmonyContainer')
        
        title = title_tag.text.strip()
        reporter = reporter_tag.text.strip()
        input_date = input_date_tag.span.text.strip()
        news = re.sub(r'\s+', ' ', news_tag.text)
        return title, reporter, input_date, news
    else:
        raise Exception("요청 결과를 가져오지 못함. %d" % res.status_code)

Overwriting daumnews.py


In [31]:
from daumnews import get_daumnews_info

In [32]:
title, reporter, input_date, news =\
                get_daumnews_info('https://news.v.daum.net/v/20200722142346971')

In [33]:
print(title, reporter, input_date, news)

"하마터면 큰일 날 뻔" 뱀 물린 80대 집배원이 살려 박종국 2020.07.22. 14:23  (괴산=연합뉴스) 박종국 기자 = 충북 증평우체국 집배원이 밭에서 일하다 뱀에 물려 쓰러져 있는 80대 할머니를 구조한 사실이 뒤늦게 알려졌다. 뱀에 물린 할머니를 구한 이주훈 집배원 [증평우체국 제공. 재판매 및 DB 금지] 22일 증평우체국에 따르면 이 우체국 소속 집배원 이주훈(45) 씨는 지난 16일 오전 11시께 업무 구역인 괴산군 사리면 소매리 도로를 지나다가 밭에 쓰러져 있는 A(88·여)를 발견했다. 이 씨는 엄지손가락을 뱀에 물렸다는 할머니의 얘기를 듣고 서둘러 손등을 묶어 독이 퍼지는 것을 막고, 119에 구조 요청했다. 괴산의 한 병원으로 이송된 A 씨는 이 씨의 신속한 조처 덕에 병원 치료를 받고 완치해 나흘 만인 지난 20일 퇴원했다. 이 씨는 "그런 상황이었다면 누구나 했을 일"이라며 "할머니가 쾌차해 무사히 귀가하셨다니 다행"이라고 말했다. pjk@yna.co.kr 


## ranking 목록 크롤링
- 제목       : ul.list_news2 > li div.cont_thumb a  :: text
- 기사링크   : ul.list_news2 > li div.cont_thumb a :: href
- 기사제공처 : ul.list_news2 > li div.cont_thumb span.info_news
- 기사요약   : ul.list_news2 > li div.cont_thumb div.desc_thumb
- 이미지링크 : ul.list_news2 > li > a.link_thumb > img :: src속성


In [11]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
# age:연령별은 형태가 다름.
url = 'https://media.daum.net/ranking/kkomkkom/'
res = requests.get(url)
if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'lxml')
#     li_list = soup.select('ul.list_news2 > li')
    li_list = soup.select('ul.list_news2 > li div.cont_thumb')
#     print(len(li_list)) # 50개
    content_list = [] # (링크, 제목, 신문사, 요약))
    for li in li_list:
#         title = li.select_one('div.cont_thumb a').text.strip()
        title = li.select_one('a').text.strip()
#         article_link = li.select_one('div.cont_thumb a').get('href')
        article_link = li.select_one('a').get('href')
#         news_paper = li.select_one('div.cont_thumb span.info_news').text.strip()
        news_paper = li.select_one('span.info_news').text.strip()
#         summary = li.select_one('div.cont_thumb div.desc_thumb').text.strip()
        summary = li.select_one('div.desc_thumb').text.strip()
#         img_link = li.select_one('a.link_thumb > img').get('src')
        content_list.append((title, article_link, news_paper, summary))#, img_link))

In [12]:
df = pd.DataFrame(content_list, columns = ['title', 'article_link', 'news_paper', 'summary'])#, 'img_link'])
df

,title,article_link,news_paper,summary


In [47]:
curr_date = datetime.now().strftime('%Y-%m-%d')
df.to_csv('news_list_{}.csv'.format(curr_date), index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# import csv
from datetime import datetime #모듈:datetime, class:datetime
url = 'https://news.daum.net'
res = requests.get(url)
if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'lxml')
    news_list = soup.select('ul.list_news2 > li > div.cont_thumb')
    # print(len(news_list))
    # print(news_list[0])
    content_list = [] # (링크, 제목, 신문사, 요약))
    for div in news_list:
        link = div.select_one('strong.tit_thumb > a').get('href').strip()
        title = div.select_one('strong.tit_thumb > a').text.strip()
        newspaper = div.select_one('strong.tit_thumb > span.info_news').text.strip()
        summary = div.select_one('div.desc_thumb').text.strip()
        content_list.append([link, title, newspaper, summary])
        # csv 파일로 쓰기
    curr_date = datetime.now().strftime('%Y-%m-%d')
    df = pd.DataFrame(content_list, columns=['링크','제목','신문사','기사요약'])
    df.to_csv('news_list_{}.csv'.format(curr_date), index=False)
    # with open('news_list_{}.csv'.format(curr_date), 'wt', encoding='UTF-8', newline='') as f:
    #     writer = csv.writer(f)
    #     writer.writerows(content_list)



In [36]:
df.head()

,링크,제목,신문사,기사요약
0,https://v.daum.net/v/20200722161259830,"임오경 ""김규봉, 선수들 모아놓고 어마무시한 말로 협박""",뉴시스,[서울=뉴시스] 정진형 문광호 기자 = 임오경 더불어민주당 의원은 22일 고(故) ...
1,https://v.daum.net/v/20200722153747772,"""박지원, 다니지도 않은 조선대 성적표로 단국대 부정입학""(종합)",뉴시스,[서울=뉴시스] 김성진 기자 = 미래통합당이 22일 박지원 국가정보원장 후보자가 조...
2,https://v.daum.net/v/20200722150143583,中 '코로나 진원지' 후베이성 수재민 1300만명..황허에도 두번째 홍수,뉴시스,[서울=뉴시스] 문예성 기자 = 중국 신종 코로나바이러스 감염증(코로나19) 최대 ...
3,https://v.daum.net/v/20200722142346971,"""하마터면 큰일 날 뻔"" 뱀 물린 80대 집배원이 살려",연합뉴스,(괴산=연합뉴스) 박종국 기자 = 충북 증평우체국 집배원이 밭에서 일하다 뱀에 물려...
4,https://v.daum.net/v/20200722160203253,"경찰, 구급차 막은 택시기사 구속영장 신청..특수폭행·업무방해",연합뉴스,(서울=연합뉴스) 임성호 기자 = 구급차를 막아 이송 중인 응급환자를 사망에 이르게...


In [37]:
df.shape

(50, 4)